In [1]:
import json
import re
import os
from typing import List, Dict, Any, Optional

import google.auth
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.colab import auth, files

In [2]:
def authenticate_google_docs():
    """Authenticate with Google Docs API and return service object."""
    try:
        auth.authenticate_user()
        creds, _ = google.auth.default()
        service = build('docs', 'v1', credentials=creds)
        print("✅ Successfully authenticated with Google Docs API")
        return service
    except Exception as e:
        print(f"❌ Authentication failed: {e}")
        raise


def create_google_doc(service, title: str) -> str:
    """Create a new Google Doc and return document ID."""
    try:
        doc = service.documents().create(body={'title': title}).execute()
        doc_id = doc['documentId']
        doc_url = f"https://docs.google.com/document/d/{doc_id}"

        print(f"📄 Created document: {title}")
        print(f"🔗 Document URL: {doc_url}")
        return doc_id
    except HttpError as e:
        print(f"❌ Failed to create document: {e}")
        raise


def update_google_doc(service, doc_id: str, requests: List[Dict[str, Any]]) -> None:
    """Update Google Doc with formatting requests."""
    if not requests:
        print("⚠️  No formatting requests to apply")
        return

    try:
        batch_size = 100
        total_batches = (len(requests) - 1) // batch_size + 1

        for i in range(0, len(requests), batch_size):
            batch = requests[i:i + batch_size]
            service.documents().batchUpdate(
                documentId=doc_id,
                body={'requests': batch}
            ).execute()

            batch_num = i // batch_size + 1
            print(f"📝 Applied batch {batch_num} of {total_batches}")

        doc_url = f"https://docs.google.com/document/d/{doc_id}"
        print(f"✅ Document updated successfully: {doc_url}")

    except HttpError as e:
        error_msg = f"Google API error: {e}"
        if hasattr(e, 'resp') and hasattr(e.resp, 'status'):
            status_code = e.resp.status
            if status_code == 403:
                error_msg += " - Permission denied. Check document sharing settings."
            elif status_code == 404:
                error_msg += " - Document not found. Check document ID."
            elif status_code == 429:
                error_msg += " - Rate limit exceeded. Try again later."

        print(f"❌ {error_msg}")
        raise

In [3]:
def parse_markdown(md_text: str) -> List[Dict[str, Any]]:
    """Convert markdown text into Google Docs API formatting requests."""
    requests = []
    index = 1  # Google Docs uses 1-based indexing
    lines = md_text.split("\n")

    for line in lines:
        if not line.strip():
            continue

        requests.extend(_process_line(line, index))
        index += len(_get_formatted_text(line))

    return requests


def _get_formatted_text(line: str) -> str:
    """Get the formatted text that will be inserted for a line."""
    line_stripped = line.strip()

    # Handle different line types
    if line.lstrip().startswith(('- ', '* ')):
        indent_level = (len(line) - len(line.lstrip())) // 2
        actual_text = line.lstrip()[2:].strip()
        return f"{'  ' * indent_level}• {actual_text}\n"
    elif line_stripped.startswith('- [ ]'):
        return f"☐ {line_stripped[5:].strip()}\n"
    elif line_stripped.startswith('- [x]'):
        return f"✓ {line_stripped[5:].strip()}\n"
    elif line_stripped.startswith('# '):
        return f"{line_stripped[2:].strip()}\n"
    elif line_stripped.startswith('## '):
        return f"{line_stripped[3:].strip()}\n"
    elif line_stripped.startswith('### '):
        return f"{line_stripped[4:].strip()}\n"
    elif line_stripped == "---":
        return "─" * 50 + "\n"
    else:
        return f"{line_stripped}\n"


def _process_line(line: str, index: int) -> List[Dict[str, Any]]:
    """Process a single line and return formatting requests."""
    requests = []
    line_stripped = line.strip()
    text_to_insert = _get_formatted_text(line)
    text_length = len(text_to_insert)

    # Insert text
    requests.append({
        "insertText": {
            "location": {"index": index},
            "text": text_to_insert
        }
    })

    # Apply styles
    requests.extend(_apply_heading_style(line_stripped, index, text_length))
    requests.extend(_apply_footer_style(line_stripped, index, text_length))
    requests.extend(_apply_bullet_formatting(line, index, text_length))
    requests.extend(_apply_mention_formatting(line, text_to_insert, index))

    return requests


def _apply_heading_style(line_stripped: str, index: int, text_length: int) -> List[Dict[str, Any]]:
    """Apply heading styles to text."""
    style_map = {
        '# ': 'HEADING_1',
        '## ': 'HEADING_2',
        '### ': 'HEADING_3'
    }

    for prefix, style in style_map.items():
        if line_stripped.startswith(prefix):
            return [{
                "updateParagraphStyle": {
                    "range": {"startIndex": index, "endIndex": index + text_length - 1},
                    "paragraphStyle": {"namedStyleType": style},
                    "fields": "namedStyleType"
                }
            }]
    return []


def _apply_footer_style(line_stripped: str, index: int, text_length: int) -> List[Dict[str, Any]]:
    """Apply footer styling to special lines."""
    footer_indicators = [
        "meeting recorded by",
        "duration",
        "---"
    ]

    if any(line_stripped.lower().startswith(indicator) or line_stripped == indicator
           for indicator in footer_indicators):
        return [{
            "updateTextStyle": {
                "range": {"startIndex": index, "endIndex": index + text_length - 1},
                "textStyle": {
                    "italic": True,
                    "fontSize": {"magnitude": 10, "unit": "PT"}
                },
                "fields": "italic,fontSize"
            }
        }]
    return []


def _apply_bullet_formatting(line: str, index: int, text_length: int) -> List[Dict[str, Any]]:
    """Apply bullet point indentation."""
    if line.lstrip().startswith(('- ', '* ')):
        indent_level = (len(line) - len(line.lstrip())) // 2
        if indent_level > 0:
            indent_pt = indent_level * 18
            return [{
                "updateParagraphStyle": {
                    "range": {"startIndex": index, "endIndex": index + text_length - 1},
                    "paragraphStyle": {
                        "indentFirstLine": {"magnitude": indent_pt, "unit": "PT"},
                        "indentStart": {"magnitude": indent_pt, "unit": "PT"}
                    },
                    "fields": "indentFirstLine,indentStart"
                }
            }]
    return []


def _apply_mention_formatting(line: str, text_to_insert: str, index: int) -> List[Dict[str, Any]]:
    """Apply bold blue formatting to @mentions."""
    requests = []

    # Calculate offset for mentions
    mention_offset = 0
    if line.lstrip().startswith(('- ', '* ')):
        indent_level = (len(line) - len(line.lstrip())) // 2
        mention_offset = len(f"{'  ' * indent_level}• ")
    elif line.strip().startswith('- [ ]') or line.strip().startswith('- [x]'):
        mention_offset = 2  # "☐ " or "✓ "

    # Find @mentions in the actual text content
    actual_text = line.lstrip()[2:].strip() if line.lstrip().startswith(('- ', '* ')) else line.strip()
    if line.strip().startswith('- [ ]') or line.strip().startswith('- [x]'):
        actual_text = line.strip()[5:].strip()

    for match in re.finditer(r"@\w+", actual_text):
        mention_start = index + mention_offset + match.start()
        mention_end = index + mention_offset + match.end()
        requests.append({
            "updateTextStyle": {
                "range": {"startIndex": mention_start, "endIndex": mention_end},
                "textStyle": {
                    "bold": True,
                    "foregroundColor": {
                        "color": {
                            "rgbColor": {"red": 0.2, "green": 0.4, "blue": 0.8}
                        }
                    }
                },
                "fields": "bold,foregroundColor"
            }
        })

    return requests

In [6]:
def load_markdown_file(file_path: Optional[str] = None) -> str:
    """Load markdown content from file or prompt for upload."""
    if file_path and os.path.exists(file_path):
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                content = f.read()
            print(f"📁 Loaded file: {file_path}")
            return content
        except Exception as e:
            print(f"❌ Error reading file {file_path}: {e}")
            raise

    # Interactive file upload
    print("📤 Please upload a markdown file:")
    uploaded = files.upload()

    if not uploaded:
        raise ValueError("No file uploaded")

    filename = list(uploaded.keys())[0]
    try:
        with open(filename, 'r', encoding='utf-8') as f:
            content = f.read()
        print(f"📁 Loaded uploaded file: {filename}")
        return content
    except Exception as e:
        print(f"❌ Error reading uploaded file: {e}")
        raise


def extract_title_from_markdown(content: str, default_title: str = "Converted Document") -> str:
    """Extract document title from first H1 heading."""
    for line in content.split('\n'):
        line = line.strip()
        if line.startswith('# '):
            title = line[2:].strip()
            return title if title else default_title
    return default_title


def get_user_choice() -> str:
    """Get user's choice for markdown source."""
    print("📋 Choose your markdown source:")
    print("1. Use example meeting notes (default)")
    print("2. Upload your own file")
    print("3. Specify file path")

    while True:
        choice = input("Enter your choice (1-3) or press Enter for default: ").strip()
        if choice in ['', '1']:
            return 'example'
        elif choice == '2':
            return 'upload'
        elif choice == '3':
            return 'path'
        else:
            print("❌ Invalid choice. Please enter 1, 2, 3, or press Enter for default.")


def load_example_markdown() -> str:
    """Load the example markdown file."""
    example_path = "example.md"

    # Check if example.md exists in current directory
    if os.path.exists(example_path):
        with open(example_path, 'r', encoding='utf-8') as f:
            content = f.read()
        print(f"📄 Loaded example file: {example_path}")
        return content
    else:
        # Fallback to embedded example if file doesn't exist
        print("📄 Using embedded example content")
        return """# Product Team Sync - May 15, 2023

## Attendees
- Sarah Chen (Product Lead)
- Mike Johnson (Engineering)
- Anna Smith (Design)
- David Park (QA)

## Agenda

### 1. Sprint Review
* Completed Features
  * User authentication flow
  * Dashboard redesign
  * Performance optimization
    * Reduced load time by 40%
    * Implemented caching solution
* Pending Items
  * Mobile responsive fixes
  * Beta testing feedback integration

### 2. Current Challenges
* Resource constraints in QA team
* Third-party API integration delays
* User feedback on new UI
  * Navigation confusion
  * Color contrast issues

### 3. Next Sprint Planning
* Priority Features
  * Payment gateway integration
  * User profile enhancement
  * Analytics dashboard
* Technical Debt
  * Code refactoring
  * Documentation updates

## Action Items
- [ ] @sarah: Finalize Q3 roadmap by Friday
- [ ] @mike: Schedule technical review for payment integration
- [ ] @anna: Share updated design system documentation
- [ ] @david: Prepare QA resource allocation proposal

## Next Steps
* Schedule individual team reviews
* Update sprint board
* Share meeting summary with stakeholders

## Notes
* Next sync scheduled for May 22, 2023
* Platform demo for stakeholders on May 25
* Remember to update JIRA tickets

---
Meeting recorded by: Sarah Chen
Duration: 45 minutes"""


def main():
    """Main execution function."""
    try:
        print("🚀 Starting markdown to Google Docs conversion...")

        # Authenticate
        service = authenticate_google_docs()

        # Get user choice for markdown source
        print("\n" + "="*50)
        choice = get_user_choice()
        print("="*50 + "\n")

        # Load markdown content based on choice
        print("📄 Loading markdown content...")

        if choice == 'example':
            markdown_content = load_example_markdown()
        elif choice == 'upload':
            markdown_content = load_markdown_file()
        elif choice == 'path':
            file_path = input("Enter the full path to your markdown file: ").strip()
            if not file_path:
                raise ValueError("No file path provided")
            markdown_content = load_markdown_file(file_path)

        if not markdown_content.strip():
            raise ValueError("Empty markdown content")

        # Extract title and create document
        doc_title = extract_title_from_markdown(markdown_content)
        print(f"\n📝 Document title: {doc_title}")

        doc_id = create_google_doc(service, doc_title)

        # Parse and apply formatting
        print("\n⚙️  Parsing markdown content...")
        requests = parse_markdown(markdown_content)
        print(f"📊 Generated {len(requests)} formatting requests")

        print("\n🎨 Applying formatting to document...")
        update_google_doc(service, doc_id, requests)

        print("\n🎉 Conversion completed successfully!")

    except Exception as e:
        print(f"\n💥 Conversion failed: {e}")
        print("Please check your file and authentication, then try again.")
        raise


# Run the main function
if __name__ == "__main__":
    main()

🚀 Starting markdown to Google Docs conversion...
✅ Successfully authenticated with Google Docs API

📋 Choose your markdown source:
1. Use example meeting notes (default)
2. Upload your own file
3. Specify file path
Enter your choice (1-3) or press Enter for default: 3

📄 Loading markdown content...
Enter the full path to your markdown file: example.md
📤 Please upload a markdown file:


Saving example.md to example.md
📁 Loaded uploaded file: example.md

📝 Document title: Product Team Sync - May 15, 2023
📄 Created document: Product Team Sync - May 15, 2023
🔗 Document URL: https://docs.google.com/document/d/1AZDp86Cvd3pH4viJBRQz7rNFRWa4eJyiZrUgz0MpGSQ

⚙️  Parsing markdown content...
📊 Generated 77 formatting requests

🎨 Applying formatting to document...
📝 Applied batch 1 of 1
✅ Document updated successfully: https://docs.google.com/document/d/1AZDp86Cvd3pH4viJBRQz7rNFRWa4eJyiZrUgz0MpGSQ

🎉 Conversion completed successfully!
